In [12]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.service import Service as ChromeService
from selenium.webdriver.support.ui import WebDriverWait
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup
import requests
import re
import numpy as np

job_titles = []
company_names = []
company_ratings = []
company_locations = []
job_pays = []
descriptions = []
posted = []
current_page = 1
url = 'https://www.indeed.com/jobs?q=Data+Science&l=Los+Angeles%2C+CA&vjk=74cadd03cb0dd918'
driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()))
driver.get(url)
pagination = driver.find_element(By.XPATH, ".//nav[contains(@role, 'navigation')]")
last_page = pagination.find_elements(By.XPATH, ".//a[contains(@data-testid, 'pagination-page-5')]")[-1].text
driver.set_page_load_timeout(10)

for page_num in range(1, int(last_page) + 11):
    container = driver.find_element(By.XPATH, ".//div[contains(@id, 'mosaic-jobResults')]")
    all_jobs = container.find_elements(By.XPATH, ".//div[contains(@class, 'job_seen_beacon')]")
    for job in all_jobs:
        title = job.find_element(By.XPATH, ".//h2[contains(@class, 'jobTitle')]").text
        try:
            location = job.find_element(By.XPATH, ".//div[contains(@class, 'company_location')]").text.split("\n")[1]
        except:
            location = job.find_element(By.XPATH, ".//div[contains(@class, 'company_location')]").text.split("\n")[0]
        company = job.find_element(By.XPATH, ".//div[contains(@class, 'company_location')]").text.split("\n")[0]
        when_posted= job.find_element(By.XPATH, ".//tr[contains(@class, 'underShelfFooter')]").text.split("\n")[-1].strip("More...")
        description = job.find_element(By.XPATH, ".//tr[contains(@class, 'underShelfFooter')]").text.split("\n")[0]
        try:
            string = job.find_element(By.XPATH, ".//div[contains(@class, 'salary-snippet-container')]").text
            listy = string.split()
            for element in listy:
                if "$" in element:
                    pay = element
        except:
            pay = "NaN"
        job_titles.append(title)
        job_pays.append(pay)
        company_names.append(company)
        descriptions.append(description)
        company_locations.append(location)
        posted.append(when_posted)
    page_num += 1

    try:
        #next = driver.find_element(By.XPATH, ".//a[contains(@aria-label, 'Next Page')]")
        next = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, ".//a[contains(@aria-label, 'Next Page')]")))
        print(next)
        next.click()
    except:
        #next = False
        print(f'failed {page_num}')
        pass

<selenium.webdriver.remote.webelement.WebElement (session="6fbb3a5c2c76b654b3146c3f3c54c931", element="f.0020F98075FA940903C9A82BAD5F8255.d.D9609954A6D9B45648AB00096A538464.e.2242")>
<selenium.webdriver.remote.webelement.WebElement (session="6fbb3a5c2c76b654b3146c3f3c54c931", element="f.0020F98075FA940903C9A82BAD5F8255.d.2AD575EB4E3C8608CD2AD91DB878EF4B.e.6008")>
<selenium.webdriver.remote.webelement.WebElement (session="6fbb3a5c2c76b654b3146c3f3c54c931", element="f.0020F98075FA940903C9A82BAD5F8255.d.2C598F1F5F4CEECA77D1DFC800217628.e.9958")>
<selenium.webdriver.remote.webelement.WebElement (session="6fbb3a5c2c76b654b3146c3f3c54c931", element="f.0020F98075FA940903C9A82BAD5F8255.d.F4EEEE614A7BF7C6784241355292A3FA.e.13790")>
<selenium.webdriver.remote.webelement.WebElement (session="6fbb3a5c2c76b654b3146c3f3c54c931", element="f.0020F98075FA940903C9A82BAD5F8255.d.C58698C099D20BE3B569872A14BE5137.e.20350")>
<selenium.webdriver.remote.webelement.WebElement (session="6fbb3a5c2c76b654b3146c3f

In [13]:
#separate into city and state
cities = []
states = []
# Parse location data
for location in company_locations:
    parts = location.split(', ')
    if len(parts) > 1:
        city = parts[0]
        state = parts[1] if len(parts) > 1 else None
        cities.append(city)
        states.append(state[:2])
    else:
        cities.append(np.nan)
        states.append(np.nan)


#make salaries integers
salaries = []
for salary in job_pays:
    if salary == "NaN" or salary == "":
        salaries.append(np.nan)
    if len(salary<= 4):
        #Then I know it is hourly rate
        
        pass
    else:
        salaries.append(float(salary.replace('$', '').replace(',', '')))


job_info = pd.DataFrame({"Title": job_titles,
              "Company": company_names,
              "Location": company_locations,
              "Job Description": descriptions,
              "When Posted": posted,
              "Pay": salaries,
              "City":cities,
               "State": states})
#try getting hybrid or remote status
def is_hybrid_remote(location):
    return 'Hybrid' in location or 'Remote' in location

# Apply the function to create a new column 'Is_Hybrid_Remote'
job_info['Is_Hybrid_Remote'] = job_info['Location'].apply(is_hybrid_remote)

In [14]:
job_info

,Title,Company,Location,Job Description,When Posted,Pay,City,State,Is_Hybrid_Remote
0,Software Developer - AI Trainer (Contract),DataAnnotation,4.5,You will work with the chatbots that we are bu...,Active 8 days ag,40.00,NaN,NaN,False
1,Data Engineer,LegalMation,"Hybrid remote in Los Angeles, CA 90071","Experience with cloud technologies, NLP, and m...",Active 4 days ag,80000.00,Hybrid remote in Los Angeles,CA,True
2,AI Training for Physics,Outlier,3.7,"Help train AI models to become more accurate, ...",Today,50.00,NaN,NaN,False
3,Associate Data Analyst - Hybrid Remote,CEDARS-SINAI,"Hybrid remote in Los Angeles, CA",Uses data quality tools to proactively identif...,Today·,45.09,Hybrid remote in Los Angeles,CA,True
4,"Data Scientist, 1+ Years of Experience",Snapchat,"Los Angeles, CA 90291",1+ years of experience in quantitative analysi...,Posted 4 days ago·,NaN,Los Angeles,CA,False
...,...,...,...,...,...,...,...,...,...
220,Data/GIS Analyst - Advanced Analytics Specialist,Hazen and Sawyer,"Los Angeles, CA 90017","Proficiency in database development, data anal...",Posted 30+ days ago·,85000.00,Los Angeles,CA,False
221,Chief AI Officer,Proper Hospitality,"Los Angeles, CA",Oversee the build-out of AI infrastructure and...,Today·,250000.00,Los Angeles,CA,False
222,Senior Manager Marketing Analytics DIG,Providence Digital Innovation Group,"Los Angeles, CA 90006",5 years Data Science/ Advanced statistical mod...,Posted 30+ days ago·,132.61,Los Angeles,CA,False
223,Principal AM Process Development Engineer,Divergent 3D,"Torrance, CA",7)Assure proper formulation and design of expe...,Posted 30+ days ago·,210000.00,Torrance,CA,False


In [15]:
#job_info.to_csv("masterpiece.csv")